In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\1.xml,9,717,4,374
1,./images\10.xml,76,166,66,129
2,./images\100.xml,103,160,79,104
3,./images\101.xml,383,586,351,465
4,./images\107.xml,205,469,94,165


In [3]:
import xml.etree.ElementTree as xet

In [4]:
filename=df['filepath'][0]
filename

'./images\\1.xml'

In [5]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./images',filename_image)
    return filepath_image

In [6]:
getFilename(filename)

'./images\\1.png'

In [7]:
image_path=list(df['filepath'].apply(getFilename))
image_path

['./images\\1.png',
 './images\\10.jpeg',
 './images\\100.jpg',
 './images\\101.jpg',
 './images\\107.jpg',
 './images\\109.jpg',
 './images\\11.jpeg',
 './images\\12.jpeg',
 './images\\13.jpeg',
 './images\\14.jpeg',
 './images\\15.jpeg',
 './images\\16.jpeg',
 './images\\18.jpeg',
 './images\\19.jpeg',
 './images\\2.jpg',
 './images\\20.jpeg',
 './images\\21.jpeg',
 './images\\22.jpeg',
 './images\\23.jpeg',
 './images\\24.jpeg',
 './images\\25.jpeg',
 './images\\26.jpeg',
 './images\\27.jpeg',
 './images\\28.jpeg',
 './images\\29.jpeg',
 './images\\3.jpg',
 './images\\30.jpeg',
 './images\\31.jpeg',
 './images\\32.jpeg',
 './images\\33.jpeg',
 './images\\34.jpeg',
 './images\\35.jpeg',
 './images\\36.jpeg',
 './images\\37.jpg',
 './images\\38.jpg',
 './images\\39.jpg',
 './images\\4.jpg',
 './images\\40.jpg',
 './images\\41.jpg',
 './images\\42.jpg',
 './images\\43.jpg',
 './images\\44.jpg',
 './images\\45.jpg',
 './images\\46.jpg',
 './images\\47.jpg',
 './images\\48.jpg',
 './imag

In [8]:
file_path = image_path[0]
file_path

'./images\\1.png'

In [9]:
img= cv2.imread(file_path)

cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
cv2.rectangle(img,(9,4),(717,374),(0,255),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [12]:
labels = df.iloc[:,1:].values

In [13]:
len(image_path)

92

In [14]:
data =[]
output=[]
for ind in range (len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    #preprocessing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 #normalization
    #normalization to labels
    xmin,xmax,ymin,ymax= labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax= ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) #normalized output
    #---------------append
    data.append(norm_load_image_arr)
    output.append(label_norm)
    

In [15]:
x=np.array(data,dtype=np.float32)
y=np.array(data,dtype=np.float32)

In [16]:
x.shape,y.shape

((92, 224, 224, 3), (92, 224, 224, 3))

In [17]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((73, 224, 224, 3), (19, 224, 224, 3), (73, 224, 224, 3), (19, 224, 224, 3))

In [18]:
from tensorflow.keras.applications import MobileNetV2, InceptionResNetV2,InceptionV3
from tensorflow.keras.layers import Dense, Dropout, Flatten,Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [19]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=True,
                                   input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=True
#.......................
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel= Dense(500,activation="relu")(headmodel)
headmodel= Dense(250,activation="relu")(headmodel)
headmodel= Dense(125,activation="relu")(headmodel)
headmodel = Dense(3,activation="sigmoid")(headmodel)
#model..................
model= Model (inputs= inception_resnet.input, outputs= headmodel)

In [20]:
#compile model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 111, 111,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 111, 111,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 109, 109,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 54, 54,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 54, 54,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 52, 52,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 56,531,239 (215.65 MB)

 Trainable params: 56,470,695 (215.42 MB)

 Non-trainable params: 60,544 (236.50 KB)

## model training

In [21]:
from tensorflow.keras.callbacks import TensorBoard

In [22]:
tfb=TensorBoard("object_detection")

In [23]:
print(x_train.shape,y_train.shape)

(73, 224, 224, 3) (73, 224, 224, 3)


In [24]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,
                    validation_data=(x_test,y_test),callbacks=[tfb])
    

Epoch 1/100


C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor']. Received: the structure of inputs=*
  warnings.warn(


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/compile_loss/mse/sub/BroadcastGradientArgs defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\ProgramData\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start

  File "C:\ProgramData\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\ProgramData\anaconda3\Lib\asyncio\windows_events.py", line 322, in run_forever

  File "C:\ProgramData\anaconda3\Lib\asyncio\base_events.py", line 641, in run_forever

  File "C:\ProgramData\anaconda3\Lib\asyncio\base_events.py", line 1986, in _run_once

  File "C:\ProgramData\anaconda3\Lib\asyncio\events.py", line